# Building Fast Queries on a CSV

In this mini project, we will imagine that we own an online laptop store and want to build a way to answer a few different business questions about our inventory.

We will use the laptops.csv file as our inventory. We uses the CSV file from the [Laptop Prices dataset on Kaggle](https://www.kaggle.com/ionaskel/laptop-prices). We changed the IDs and made the prices integers. 

Here is a brief description of the rows:
- ID: A unique identifier for the laptop.
- Company: The name of the company that produces the laptop.
- Product: The name of the laptop.
- TypeName: The type of laptop.
- Inches: The size of the screen in inches.
- ScreenResolution: The resolution of the screen.
- CPU: The laptop CPU.
- RAM: The amount of RAM in the laptop.
- Memory: The size of the hard drive.
- GPU: The graphics card name.
- OpSys: The name of the operating system.
- Weight: The laptop weight.
- Price: The price of the laptop.

The goal of this project is to create a class that represents our inventory. The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.


Let's have a look the headers and first couple of rows of data.

In [3]:
import csv
with open('laptops.csv', encoding='UTF-8') as file:
    read_file = list(csv.reader(file))
    header = read_file[0]
    rows = read_file[1:]
    print(header)
    print(rows[:6])

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
[['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339'], ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'], ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'], ['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537'], ['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB 

Hear are some queries that we will want to answer:
- Given a laptop id, find the corresponding data
- Given an amount of money, find whether there are two laptops whose total price is that given amount.
- Identify all laptops whose price falls within a given budget.

To start, let's create a class with constructor takes the name of the CSV file as argument and then read the rows contained it.

In [4]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename, encoding='utf-8') as file:
            data = list(csv.reader(file))
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row_len = len(row)
                row[row_len - 1] = int(row[row_len - 1])

Test our class by creating an instance of `Inventory` using `laptops.csv` as argument.

In [5]:
inventory = Inventory('laptops.csv')
print(inventory.header)
print(len(inventory.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303


The first thing that we will implement is a way to look up a laptop from a given identifier. In this way, when a customer comes to our store with a purchase slip, we can quickly identify the laptop to which it corresponds.

In [11]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename, encoding='utf-8') as file:
            data = list(csv.reader(file))
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row_len = len(row)
                row[row_len - 1] = int(row[row_len - 1])
        
    def get_laptop_from_id(self, laptop_id): 
        for row in self.rows:
            if laptop_id in row:
                return row
        return None

Testing modified class by creating an instance of `Inventory` using `laptops.csv` as argument.

In [13]:
inventory = Inventory('laptops.csv')
item1 = inventory.get_laptop_from_id('3362737')
print(item1)
item2 = inventory.get_laptop_from_id('3362736')
print(item2)

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## Improving Id Lookups

By preprocessing the data, we can make this code more efficiently. Using a set, we can check in constant time whether a given identifier exists. However, we don't just want to know if it exists, we also want to retrieve the remaining row information. Therefore, we will use a dictionary instead of set. Dictionaries have the same fast lookup properties that sets have, but allow us to associate values to the keys.

We can preprocess data and create the dictionary in the `__init()__` method.

In [14]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename, encoding='utf-8') as file:
            data = list(csv.reader(file))
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row_len = len(row)
                row[row_len - 1] = int(row[row_len - 1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
        
    def get_laptop_from_id(self, laptop_id): 
        for row in self.rows:
            if laptop_id in row:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None

Let's test if the new method `get_laptop_from_id_fast()` works properly.

In [16]:
inventory = Inventory('laptops.csv')
item1 = inventory.get_laptop_from_id_fast('3362737')
print(item1)
item2 = inventory.get_laptop_from_id_fast('3362736')
print(item2)

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## Comparing Performance

The `get_laptop_from_id()` method has time complexity _O(R)_ where R is the number of rows. In contrast, the new implementation as time complexity _O(1)_. It does so by using more memory to store the `self.id_to_row` dictionary and using a bit more time creating an instance (because it needs to create the dictionary).

In [17]:
import time
import random
ids = [ str(random.randint(1000000, 9999999)) for _ in range(10000) ]
inventory = Inventory('laptops.csv')

total_time_no_dict = 0 # this will aggregate the times of calling get_laptop_from_id().
total_time_dict = 0 # this will aggregate the times of calling get_laptop_from_id_fast().

for id_str in ids:
    start = time.time()
    inventory.get_laptop_from_id(id_str)
    end = time.time()
    total_time_no_dict = end - start
    
    start = time.time()
    inventory.get_laptop_from_id_fast(id_str)
    end = time.time()
    total_time_dict = end - start

print("Total time without dictionary: ", total_time_no_dict)
print("Total time with dictionary: ", total_time_dict)


Total time without dictionary:  0.00033736228942871094
Total time with dictionary:  7.152557373046875e-07


## Two Laptop Promotion

Sometimes, our store offers a promotion where we give a gift card. A customer can use the gift to buy up to two laptops. To avoid having to keep track of what was already spent, the gift card has a single time usage. This means that, even if there is leftover money, it cannot be used anymore.

Let's imagine there are three laptops prices \\$1,339, \\$898 and \\$575. Say we offered a gift card of \\$2,500. Since a customer can buy, at most, two laptops with a gift card, the maximum they can spend is \\$2,237 (\\$1,339 plus \\$898). Therefore, they might feel cheated because, no matter how they spend their gift card, they cannot spend the full \\$2,500.

To avoid makes customer feel cheated, we want to make sure that there is at least one way to spend it in full. In other words, before issuing a gift card for _D_ dollars, we want to make sure that either there is a laptop that costs exactly _D_ dollars or two laptops whose costs add up to precisely _D_ dollars.


In [20]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename, encoding='utf-8') as file:
            data = list(csv.reader(file))
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row_len = len(row)
                row[row_len - 1] = int(row[row_len - 1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
        
    def get_laptop_from_id(self, laptop_id): 
        for row in self.rows:
            if laptop_id in row:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for i in range(len(self.rows)):
            if self.rows[i][12] == dollars:
                return True
            for j in range(i, len(self.rows)):
                if self.rows[i][12] + self.rows[j][12] == dollars:
                    return True
        return False

Let's test the `check_promotion_dollars()` function.

Call the function by giving `1000` as argument should find a solution.
Call the function by giving `442` as argument should not find a solution.

In [21]:
inventory = Inventory('laptops.csv')
promo1 = inventory.check_promotion_dollars(1000)
print(promo1)
promo2 = inventory.check_promotion_dollars(442)
print(promo2)

True
False


## Optimizing Laptop Promotion

Since we only care about whether or not there is a solution, we can store all laptops prices in a set when we initialize the inventory. Then we check in constant time whether there is a laptop with a given price.

In [25]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename, encoding='utf-8') as file:
            data = list(csv.reader(file))
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row_len = len(row)
                row[row_len - 1] = int(row[row_len - 1])
        
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[12])
        
    def get_laptop_from_id(self, laptop_id): 
        for row in self.rows:
            if laptop_id in row:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for i in range(len(self.rows)):
            if self.rows[i][12] == dollars:
                return True
            for j in range(i, len(self.rows)):
                if self.rows[i][12] + self.rows[j][12] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        
        for price1 in self.prices:
            price2 = dollars - price1
            if price2 in self.prices:
                return True
            
        return False

Let's test our `check_promotion_dollars_fast()` function by giving `1000` and `442` as argument.
Giving `1000` should find a solution.
Giving `442` should not find a solution.

In [26]:
inventory = Inventory('laptops.csv')
promo1 = inventory.check_promotion_dollars_fast(1000)
print(promo1)
promo2 = inventory.check_promotion_dollars_fast(442)
print(promo2)

True
False


## Comparing Promotion Functions

Let's compare the performance of the last two functions that we wrote.

In [28]:
prices = [ random.randint(100, 5000) for _ in range(100) ]
inventory = Inventory('laptops.csv')
total_time_no_set = 0
total_time_set = 0
for price in prices:
    start = time.time()
    inventory.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set = end - start
    
    start = time.time()
    inventory.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set = end - start
    
print("Total time with no set: ", total_time_no_set)
print("Total time with set: ", total_time_set)

Total time with no set:  0.008693218231201172
Total time with set:  2.86102294921875e-06


## Finding Laptops Within a Budget

We are going to leverage binary search to write a method that efficiently answers the query: Given a budget of _D_ dollars, find all laptops whose price it at most _D_.

We are going to sort all laptops by price so that we can use binary search to identify the first laptop in the sorted list with a price larger than _D_. We need to make sure that our binary search finds the _first one_ on the list. Then, the result of the query will consist of all laptops whose index in the sorted list is smaller than the index of the first laptop whose price is higher than the budget _D_.

In [68]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename, encoding='utf-8') as file:
            data = list(csv.reader(file))
            self.header = data[0]
            self.rows = data[1:]
            for row in self.rows:
                row_len = len(row)
                row[row_len - 1] = int(row[row_len - 1])
        
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[12])
            
        self.rows_by_price = sorted(self.rows, key=self.row_price)
        
    def get_laptop_from_id(self, laptop_id): 
        for row in self.rows:
            if laptop_id in row:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for i in range(len(self.rows)):
            if self.rows[i][12] == dollars:
                return True
            for j in range(i, len(self.rows)):
                if self.rows[i][12] + self.rows[j][12] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        
        for price1 in self.prices:
            price2 = dollars - price1
            if price2 in self.prices:
                return True
            
        return False
    
    def row_price(self, row):
        return row[-1]
    
    def find_first_laptop_more_expensive(self, budget):
        range_start = 0
        range_end = len(self.rows_by_price) - 1
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2
            price = int(self.rows_by_price[range_middle][-1])
            if price >= budget:
                range_end =  range_middle
            elif price < budget:
                range_start = range_middle + 1
        
        if int(self.rows_by_price[range_start][-1]) == budget:
            return range_start + 1
        elif int(self.rows_by_price[range_start][-1]) > budget:
            return range_start
        else: 
            return -1
    

Let's test our `find_first_laptop_more_expensive()` function.
Call the function by giving `1000` as argument should find the index `683`.
Call the function by giving `10000` as argument should not find a solution and return `-1`.

In [69]:
inventory = Inventory('laptops.csv')
find1 = inventory.find_first_laptop_more_expensive(1000)
print(find1)
find2 = inventory.find_first_laptop_more_expensive(10000)
print(find2)

683
-1
